In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# file = open('data\kml\export.kml')
# xml_data = file.read()

# print(xml_data)

# file.close()

with open('data\kml\My Places_dec_9_22.kml') as file:

    xml_data = file.read()

    # print (xml_data)

In [2]:
# Initializing soup variable
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])

## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text

    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text
# print(foul)
# print(fop)
    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)


    
        


    

    # for polygon in layer.find_all('Placemark'):
        #     polygon_name = polygon.name.text
        #     print(polygon_name)
        #     for foul in polygon.find_all('SimpleData'):
        #         foul_name = foul.text
        #         print(foul_name)
        #         for fop in foul.find_all_next('SimpleData'):
        #             fop_name = fop.text
        #             print(fop_name)
        #             df = df.append({'name': name, 'layer': layer_name, 'polygon': polygon_name, 'foul': foul_name, 'fop': fop_name}, ignore_index=True)

C:\Users\Justin\AppData\Local\Temp\ipykernel_11152\1768877062.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_11152\1768877062.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_11152\1768877062.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_11152\1768877062.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [3]:
list[56]


# df['foul'] = df['foul'].str.replace(' ', ',')
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

df




,field,foul,fop
0,1. Beal City,"-84.9127615,43.6741539,0 -84.91275210000001,43...","-84.9127615,43.6741539,0 -84.9115486,43.674159..."
1,1. Buchanan,"-86.37097850000001,41.8239409,0 -86.3717456,41...","-86.37097850000001,41.8239409,0 -86.3700572000..."
2,1. Grosse Pointe Woods University Liggett,"-82.90074989999999,42.4260393,0 -82.9002148,42...","-82.90074989999999,42.4260393,0 -82.8997751,42..."
3,10. Jackson Lumen Christi,"-84.4602665,42.2219816,0 -84.4603248,42.222842...","-84.4602665,42.2219816,0 -84.4591654,42.222017..."
4,10. Kalamazoo Christian,"-85.6134503,42.273406,0 -85.61346399999999,42....","-85.6134503,42.273406,0 -85.6145228,42.2734125..."
...,...,...,...
247,Whitehall High School,"-86.33278559999999,43.4045055,0 -86.3329868,43...","-86.33278559999999,43.4045055,0 -86.3316323,43..."
248,Whitmore Lake High School,"-83.7708313,42.3876304,0 -83.7700119,42.387014...","-83.7708313,42.3876304,0 -83.7716762,42.387022..."
249,Whittemore-Prescott High School,"-83.8844484,44.2293306,0 -83.88555700000001,44...","-83.8844484,44.2293306,0 -83.8844618,44.230166..."
250,Williamston High School,"-84.28086999999999,42.6987035,0 -84.2809585,42...","-84.28086999999999,42.6987035,0 -84.2797126,42..."


In [35]:
df['home_lat'] = df['foul'].str.split(',').str[0]
df['home_lon'] = df['foul'].str.split(',').str[1]
df['home_pt'] = df['home_lat'].str.cat(df['home_lon'], sep=',')


# for i in range(len(df['home_lat'])):
#     # df['home_plate'][i] = df['home_lat'][i].replace(' ', ''), df['home_lon'][i].replace(' ', '')
#     df['home_tup'][i] = df['home_lat'][i].split(',').df['home_lon'][i]

df
df.to_csv('data/coord_export.csv', index=False)

### Working Above !!!

In [6]:
 ## Start messing around with calculating area from coordinate strings in dataframe

from area import area
from shapely import geometry

df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)
df

df.to_csv('data/coord_export_after_calc.csv', index=False)

In [7]:
## Drop and rows with 0 because it screws up with geometry creator
# df = df[(df != 0).all(1)]

### Make wrt strings for each polygon

df['foul_wkt'] = df['foul'].apply(lambda x: 'POLYGON( (' + x + '))')
df['fop_wkt'] = df['fop'].apply(lambda x: 'POLYGON( (' + x + '))')


df.head()

,field,foul,fop,home_lat,home_lon,foul_area,fop_area,foul_ft,fop_ft,foul_wkt,fop_wkt
0,1. Beal City,"-84.9127615,43.6741539,0 -84.91275210000001,43...","-84.9127615,43.6741539,0 -84.9115486,43.674159...",-84.9127615,43.6741539,2387.453607,8313.729768,25698.311886,89488.155852,"POLYGON( (-84.9127615,43.6741539,0 -84.9127521...","POLYGON( (-84.9127615,43.6741539,0 -84.9115486..."
1,1. Buchanan,"-86.37097850000001,41.8239409,0 -86.3717456,41...","-86.37097850000001,41.8239409,0 -86.3700572000...",-86.37097850000001,41.8239409,2105.178908,8219.710003,22659.935253,88476.136505,"POLYGON( (-86.37097850000001,41.8239409,0 -86....","POLYGON( (-86.37097850000001,41.8239409,0 -86...."
2,1. Grosse Pointe Woods University Liggett,"-82.90074989999999,42.4260393,0 -82.9002148,42...","-82.90074989999999,42.4260393,0 -82.8997751,42...",-82.90074989999999,42.4260393,2079.959303,7721.998424,22388.473941,83118.818831,"POLYGON( (-82.90074989999999,42.4260393,0 -82....","POLYGON( (-82.90074989999999,42.4260393,0 -82...."
3,10. Jackson Lumen Christi,"-84.4602665,42.2219816,0 -84.4603248,42.222842...","-84.4602665,42.2219816,0 -84.4591654,42.222017...",-84.4602665,42.2219816,1904.292945,8550.033157,20497.618836,92031.701897,"POLYGON( (-84.4602665,42.2219816,0 -84.4603248...","POLYGON( (-84.4602665,42.2219816,0 -84.4591654..."
4,10. Kalamazoo Christian,"-85.6134503,42.273406,0 -85.61346399999999,42....","-85.6134503,42.273406,0 -85.6145228,42.2734125...",-85.6134503,42.273406,2223.572377,8566.263773,23934.310711,92206.406626,"POLYGON( (-85.6134503,42.273406,0 -85.61346399...","POLYGON( (-85.6134503,42.273406,0 -85.6145228,..."


In [19]:
df['home_plate'] = (df['home_lat'], df['home_lon'])

df.head()


# df.info()


ValueError: Length of values (2) does not match length of index (252)

### Saturday. everything working to this point

Next I need to get the wkt strings into shapely polygons

# saving at 1:30 pm Sat

In [8]:
from shapely import wkt

### Make shapely polygons for each polygon


df['fop_shapely'] = df['fop_wkt'].apply(lambda x: wkt.loads(x))
df['foul_shapely'] = df['foul_wkt'].apply(lambda x: wkt.loads(x))
df.head()

ParseException: Expected number but encountered ','


WKTReadingError: Could not create geometry because of errors while reading input.

In [ ]:
import shapely
from shapely import geometry
from shapely import wkt
from shapely.geometry import Polygon

# poly = geometry.Polygon(df['foul_poly'][1])
# pt = wkt.loads(df['home_lat'][1] + ' ' + df['home_lon'][1])

# # The point is in the polygon, so the distance will always be 0.0
# poly.distance(pt)  # 0.0

# # The distance is calculated to the nearest boundary
# poly.boundary.distance(pt)  # 4.47213595499958

# # or exterior ring
# poly.exterior.distance(pt)  # 4.47213595499958

In [ ]:
# Initializing soup variable
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon


print(len(list))
# print(list)

# print(len(folders))
# print(folders)

### 
df = pd.DataFrame(columns=['field_name','foul_coords','fop_coords'])
df

for i in list:

    field_name = layers.find('name').text
    foul = layers.find_all('coordinates')[0].text
    fop = layers.find_all('coordinates')[1].text

    row = {
            'field_name': field_name,
            'foul_coords': foul,
            'fop_coords': fop
        }
        
    df = df.append(row, ignore_index=True)
    # print(f'Appending row %s of %s' % (index+1, items_length))




# print(field_name)
# print(foul)
# print(fop)

In [ ]:
df.head()

In [ ]:
# Initializing soup variable
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
layers = soup.Document.Folder.Folder.Placemark
polygons = soup.Document.Folder.Placemark.Polygon

# Creating column for table
df = pd.DataFrame(columns=['field', 'foul', 'fop', 'home_plate'])

# Iterating through item tag and extracting elements
f_names = folders.find_all('name')
name_length = len(f_name)

all_coords = soup.find_all('coordinates')
coords_length = len(all_coords)

# print(coords_length)
# print(name_length)
# print(f_names[0:10])
# print(all_coords[0:10])

names_df = pd.DataFrame(f_names)

names_df.to_csv('export_name_list.csv')



In [ ]:
def parse_xml(xml_data):
  # Initializing soup variable
    soup = BeautifulSoup(xml_data, 'xml')
    
    folders = soup.Document.Folder.Folder
    layers = soup.Document.Folder.Folder.Placemark
    polygons = soup.Document.Folder.Placemark.Polygon

  # Creating column for table
    df = pd.DataFrame(columns=['field', 'foul', 'fop', 'home_plate'])

  # Iterating through item tag and extracting elements
    all_items = soup.Folder.find_all('Folder')
    items_length = len(all_items)
    
    for index, item in enumerate(all_items):
        field_name = folders.find('name').text
        foul_coord = polygons.find('coordinates').text
        fop_coords = layers.find('coordinates').find_next('coordinates').contents[0]


       # Adding extracted elements to rows in table
        row = {
            'field': field_name,
            'foul': fop_coords,
            'fop': fop_coords
        }
    

        df = df.append(row, ignore_index=True)
        print(f'Appending row %s of %s' % (index+1, items_length))

    return df

In [ ]:
df = parse_xml(xml_data)
df.to_csv('news.csv')